In [1]:
DATASET_FILE = "../dataset/normal.csv"
MODEL_FILE = "../models/anomaly.pkl"

CONTAMINATION = 0.1
RAND = 42
N_ESTIMATORS = 100

In [2]:
import os
os.makedirs("../models", exist_ok=True)

In [3]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv(DATASET_FILE)
df.head()

,articulation_rate,neutral,intensity_std,formant_f1_mean,formant_f1_std,talk_time_mean,pauses_num,talk_time_std,pauses_total_duration,formant_f2_std,...,zcr_std,voiced_ratio,sad,f0_std,spectral_entropy_std,zcr_mean,jitter,spectral_centroid_std,hnr_std,valence
0,5.789474,3.238715,8.549604,322.052499,76.101319,0.0,0,0.00,0.00,218.863907,...,930.682312,0.997361,-1.318469,67.318468,0.094397,1307.124011,11.046585,382.812392,29.666108,0.583235
1,4.651163,2.055671,13.868460,299.424224,61.952174,0.0,0,0.00,0.00,104.326814,...,759.930555,1.000000,-0.148386,121.328617,0.103551,1336.309524,17.526502,283.896589,20.761109,0.384466
2,5.813953,1.545099,12.035212,316.359242,50.179401,0.0,0,0.00,0.00,63.884821,...,587.751329,1.000000,-0.882056,58.374491,0.092568,1340.476190,12.439938,222.598806,5.733600,0.391351
3,5.862069,3.594836,19.424257,316.880409,120.592801,1.4,1,0.64,0.28,282.933925,...,832.108355,0.990506,-3.255889,116.716242,0.163133,1043.934599,16.577902,371.978580,8.896209,0.525628
4,5.769231,1.126917,13.030559,375.552980,99.103323,0.0,0,0.00,0.00,201.868310,...,594.957956,0.986188,-1.953436,106.443649,0.084982,1488.351750,22.227037,266.217842,36.435997,0.508493


In [5]:
df.shape

(1294, 35)

In [6]:
df.dtypes

articulation_rate         float64
neutral                   float64
intensity_std             float64
formant_f1_mean           float64
formant_f1_std            float64
talk_time_mean            float64
pauses_num                  int64
talk_time_std             float64
pauses_total_duration     float64
formant_f2_std            float64
pauses_mean_duration      float64
f0_mean                   float64
spectral_rolloff_std      float64
formant_f2_mean           float64
speaking_rate             float64
shimmer                   float64
hnr_mean                  float64
arousal                   float64
intensity_mean            float64
positive                  float64
spectral_centroid_mean    float64
other                     float64
angry                     float64
spectral_entropy_mean     float64
spectral_rolloff_mean     float64
zcr_std                   float64
voiced_ratio              float64
sad                       float64
f0_std                    float64
spectral_entro

In [7]:
df.isnull().sum()

articulation_rate         0
neutral                   0
intensity_std             0
formant_f1_mean           0
formant_f1_std            0
talk_time_mean            0
pauses_num                0
talk_time_std             0
pauses_total_duration     0
formant_f2_std            0
pauses_mean_duration      0
f0_mean                   0
spectral_rolloff_std      0
formant_f2_mean           0
speaking_rate             0
shimmer                   0
hnr_mean                  0
arousal                   0
intensity_mean            0
positive                  0
spectral_centroid_mean    0
other                     0
angry                     0
spectral_entropy_mean     0
spectral_rolloff_mean     0
zcr_std                   0
voiced_ratio              0
sad                       0
f0_std                    0
spectral_entropy_std      0
zcr_mean                  0
jitter                    0
spectral_centroid_std     0
hnr_std                   0
valence                   0
dtype: int64

In [8]:
np.isinf(df).sum()

articulation_rate         0
neutral                   0
intensity_std             0
formant_f1_mean           0
formant_f1_std            0
talk_time_mean            0
pauses_num                0
talk_time_std             0
pauses_total_duration     0
formant_f2_std            0
pauses_mean_duration      0
f0_mean                   0
spectral_rolloff_std      0
formant_f2_mean           0
speaking_rate             0
shimmer                   0
hnr_mean                  0
arousal                   0
intensity_mean            0
positive                  0
spectral_centroid_mean    0
other                     0
angry                     0
spectral_entropy_mean     0
spectral_rolloff_mean     0
zcr_std                   0
voiced_ratio              0
sad                       0
f0_std                    0
spectral_entropy_std      0
zcr_mean                  0
jitter                    0
spectral_centroid_std     0
hnr_std                   0
valence                   0
dtype: int64

In [9]:
df.describe()

,articulation_rate,neutral,intensity_std,formant_f1_mean,formant_f1_std,talk_time_mean,pauses_num,talk_time_std,pauses_total_duration,formant_f2_std,...,zcr_std,voiced_ratio,sad,f0_std,spectral_entropy_std,zcr_mean,jitter,spectral_centroid_std,hnr_std,valence
count,1294.000000,1294.000000,1294.000000,1294.000000,1294.000000,1294.000000,1294.000000,1294.000000,1294.000000,1294.000000,...,1294.000000,1294.000000,1294.000000,1294.000000,1294.000000,1294.000000,1294.000000,1294.000000,1294.000000,1294.000000
mean,5.679959,2.871197,13.268398,366.439418,90.430391,0.344712,0.648377,0.098053,0.292697,214.208873,...,907.311175,0.973521,-1.765192,115.916758,0.108709,1485.893990,29.116547,416.615842,22.766115,0.422613
std,1.644988,1.258592,6.036725,114.568184,42.719765,0.810638,1.343257,0.314440,0.860384,93.068931,...,393.880253,0.086903,1.274666,34.254746,0.063870,638.018675,16.372254,164.714796,13.312791,0.112821
min,0.000000,-1.226820,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,-4.582433,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.043840
25%,5.041278,2.064095,9.868760,329.561768,69.825806,0.000000,0.000000,0.000000,0.000000,165.646496,...,643.053101,0.981851,-2.626445,94.542846,0.069887,1119.907407,17.046744,308.828232,12.504205,0.343418
50%,5.863133,2.967069,13.117504,371.805055,87.679565,0.000000,0.000000,0.000000,0.000000,218.801046,...,909.474845,0.992188,-1.910328,117.152108,0.092971,1539.052308,27.593248,403.215432,24.327062,0.415584
75%,6.626506,3.775353,17.505988,414.925722,108.786933,0.370000,1.000000,0.000000,0.300000,268.029383,...,1159.071510,1.000000,-1.125287,138.747080,0.118730,1891.718254,39.593393,512.004397,32.728924,0.491127
max,11.764706,5.701880,28.424267,838.439343,240.951016,11.210000,17.000000,2.725000,13.350000,612.641068,...,2409.499362,1.000000,4.492037,195.095220,0.380509,4248.888889,107.012885,1619.059564,49.378506,0.885448


In [10]:
corr = df.corr().abs()
corr

,articulation_rate,neutral,intensity_std,formant_f1_mean,formant_f1_std,talk_time_mean,pauses_num,talk_time_std,pauses_total_duration,formant_f2_std,...,zcr_std,voiced_ratio,sad,f0_std,spectral_entropy_std,zcr_mean,jitter,spectral_centroid_std,hnr_std,valence
articulation_rate,1.000000,0.061481,0.119012,0.123952,0.153237,0.067302,0.008417,0.005804,0.086772,0.135457,...,0.000791,0.062589,0.023698,0.022979,0.149195,0.003306,0.037804,0.012726,0.063889,0.101504
neutral,0.061481,1.000000,0.351396,0.248050,0.244662,0.024405,0.062483,0.006080,0.058698,0.294146,...,0.302911,0.039831,0.048832,0.330665,0.103776,0.039397,0.146045,0.275852,0.000651,0.130484
intensity_std,0.119012,0.351396,1.000000,0.061802,0.089176,0.277049,0.421808,0.204421,0.295395,0.111833,...,0.420657,0.068520,0.042578,0.355752,0.440169,0.201028,0.152324,0.464326,0.265391,0.197941
formant_f1_mean,0.123952,0.248050,0.061802,1.000000,0.658603,0.021428,0.045997,0.014741,0.064636,0.508518,...,0.023033,0.076607,0.138049,0.084250,0.050585,0.134973,0.111112,0.042370,0.111956,0.015267
formant_f1_std,0.153237,0.244662,0.089176,0.658603,1.000000,0.004621,0.025838,0.027400,0.092238,0.624997,...,0.049360,0.119856,0.079310,0.014394,0.073409,0.212343,0.245909,0.077726,0.111998,0.014371
talk_time_mean,0.067302,0.024405,0.277049,0.021428,0.004621,1.000000,0.228633,0.450055,0.194571,0.093264,...,0.187444,0.054340,0.000522,0.032555,0.087866,0.037997,0.041382,0.203694,0.078237,0.039442
pauses_num,0.008417,0.062483,0.421808,0.045997,0.025838,0.228633,1.000000,0.350080,0.621014,0.113369,...,0.203624,0.080573,0.024408,0.144885,0.200194,0.061876,0.074209,0.270225,0.056468,0.057626
talk_time_std,0.005804,0.006080,0.204421,0.014741,0.027400,0.450055,0.350080,1.000000,0.282548,0.088938,...,0.153862,0.040920,0.004979,0.011853,0.078992,0.009994,0.045480,0.160202,0.029824,0.005254
pauses_total_duration,0.086772,0.058698,0.295395,0.064636,0.092238,0.194571,0.621014,0.282548,1.000000,0.124138,...,0.136159,0.309730,0.051485,0.103834,0.205908,0.032372,0.005595,0.260369,0.013991,0.053119
formant_f2_std,0.135457,0.294146,0.111833,0.508518,0.624997,0.093264,0.113369,0.088938,0.124138,1.000000,...,0.259009,0.146307,0.091412,0.036851,0.098170,0.072436,0.133769,0.277144,0.009133,0.054355


In [11]:
high_corr = np.where(corr > 0.8)
high_corr_pairs = [
    (corr.index[x], corr.columns[y], corr.iloc[x, y])
    for x, y in zip(*high_corr)
    if x != y and x < y
]
for x, y, val in high_corr_pairs:
    print(x, y, val)

spectral_rolloff_std spectral_centroid_std 0.8643946531411619
hnr_mean hnr_std 0.81205342887506
spectral_centroid_mean spectral_rolloff_mean 0.9666530352756465
spectral_centroid_mean zcr_mean 0.9004792252704926
spectral_entropy_mean spectral_rolloff_mean 0.8192445020808364
spectral_rolloff_mean zcr_mean 0.8222793907864154
zcr_std spectral_centroid_std 0.8447710842446222


In [12]:
HIGH_CORR_TO_DROP = [
    "spectral_centroid_std",
    "hnr_std",
    "spectral_rolloff_mean",
    "zcr_mean",
]

df_reduced = df.drop(columns=HIGH_CORR_TO_DROP)
df_reduced.shape

(1294, 31)

In [13]:
sorted_cols = sorted(df_reduced.columns)
df_sorted = df_reduced[sorted_cols]

In [14]:
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("isolation_forest", IsolationForest(
        contamination=CONTAMINATION,
        random_state=RAND,
        n_estimators=N_ESTIMATORS,
    ))
])

pipeline.fit(df_sorted)

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('scaler', ...), ('isolation_forest', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"copy copy: bool, default=TrueIf False, try to avoid a copy and do inplace scaling instead.This is not guaranteed to always work inplace; e.g. if the data isnot a NumPy array or scipy.sparse CSR matrix, a copy may still bereturned.",True
,"with_mean with_mean: bool, default=TrueIf True, center the data before scaling.This does not work (and will raise an exception) when attempted onsparse matrices, because centering them entails building a densematrix which in common use cases is likely to be too large to fit inmemory.",True
,"with_std with_std: bool, default=TrueIf True, scale the data to unit variance (or equivalently,unit standard deviation).",True
,"n_estimators n_estimators: int, default=100The number of base estimators in the ensemble.",100
,"max_samples max_samples: ""auto"", int or float, default=""auto""The number of samples to draw from X to train each base estimator.- If int, then draw `max_samples` samples.- If float, then draw `max_samples * X.shape[0]` samples.- If ""auto"", then `max_samples=min(256, n_samples)`.If max_samples is larger than the number of samples provided,all samples will be used for all trees (no sampling).",'auto'
,"contamination contamination: 'auto' or float, default='auto'The amount of contamination of the data set, i.e. the proportionof outliers in the data set. Used when fitting to define the thresholdon the scores of the samples.- If 'auto', the threshold is determined as in the original paper.- If float, the contamination should be in the range (0, 0.5]... versionchanged:: 0.22 The default value of ``contamination`` changed from 0.1 to ``'auto'``.",0.1
,"max_features max_features: int or float, default=1.0The number of features to draw from X to train each base estimator.- If int, then draw `max_features` features.- If float, then draw `max(1, int(max_features * n_features_in_))` features.Note: using a float number less than 1.0 or integer less than number offeatures will enable feature subsampling and leads to a longer runtime.",1.0


In [15]:
anomalies = pipeline.predict(df_sorted)
anomalies, sum(anomalies)

(array([1, 1, 1, ..., 1, 1, 1], shape=(1294,)), np.int64(1034))

In [16]:
pipeline.decision_function(df_sorted.sample(1))

array([0.1345792])

In [17]:
import joblib
joblib.dump(pipeline, MODEL_FILE)

['../models/anomaly.pkl']